In [1]:
import cv2

In [2]:
####### Import all necessity function for Deep Learning ########
import os
import cv2
import random 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, PReLU, ELU, BatchNormalization, Dropout, Conv2D, MaxPool2D, MaxPool3D, Flatten
from tensorflow.keras.initializers import HeNormal, HeUniform, GlorotNormal, GlorotUniform
from tensorflow.keras.activations import relu, sigmoid, softmax, swish
from tensorflow.keras.regularizers import L1, L2, L1L2
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy, MSE, MAE, Huber
from tensorflow.keras.optimizers import SGD, Adagrad, Adadelta, RMSprop, Adam, Adamax, Nadam

In [3]:
directory = r'C:\Users\DELL\Desktop\train'

#now we want to extract the cat and dog folder

data = []
categories = ['with_mask', 'without_mask']

for i in categories:
    
    folder_path = os.path.join(directory,i)
    
    for image in os.listdir(folder_path):
        label = categories.index(i)
        filepath_ = os.path.join(folder_path, image)
        img_arr_  = cv2.imread(filepath_)
        img_arr_  = cv2.resize(img_arr_, (224,224))
        data.append([img_arr_, label])


random.shuffle(data)


In [4]:
######## import all necessity functions ##########
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from imblearn.over_sampling import RandomOverSampler, BorderlineSMOTE, SVMSMOTE, ADASYN
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor, NearestNeighbors
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier, SGDRegressor
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, VotingClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier, RandomForestRegressor, StackingRegressor, VotingRegressor
from xgboost import XGBClassifier, XGBRegressor, XGBRegressor
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, classification_report, silhouette_score, r2_score, mean_squared_error, mean_absolute_error

In [5]:
X = []
y = []

for features, label in data:
    X.append(features)
    y.append(label)

import numpy as np
X = np.array(X)
y = np.array(y)

X = X/255

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [6]:
from tensorflow.keras.applications import VGG16

VGG_ = VGG16()

VGG_.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
VGG16_ = VGG16(include_top = False, input_shape = X.shape[1:])

model = Sequential()
for layer in VGG16_.layers:
    model.add(layer)

for layer in VGG16_.layers:
    layer.trainable = False



model.add(Flatten())
model.add(Dense(units =  1024 , activation = 'relu', kernel_initializer = HeNormal()))
model.add(Dropout(0.5))
model.add(Dense(units =  512 , activation = 'relu', kernel_initializer = HeNormal()))
model.add(Dropout(0.5))
model.add(Dense(units =  128 , activation = 'relu', kernel_initializer = HeNormal()))
model.add(Dropout(0.5))

model.add(Dense(units = 1, activation = 'sigmoid'))
model.summary()

model.compile(optimizer = 'Adam', loss = BinaryCrossentropy(), metrics = ['accuracy'])
history_ = model.fit(x = X_train, y = y_train, validation_data = (X_test, y_test), epochs = 3, batch_size = 128, verbose = 1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [ ]:
def detect_face_mask(img):
    y_pred = model.predict(img.reshape(1, 224, 224, 3))
    if y_pred[0] > 0.5:
        return 1
    else:
        return 0

In [ ]:
def draw_label(img, text, pos, bg_color):
    text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_COMPLEX, 1, cv2.FILLED)
    end_x = pos[0] + text_size[0][0] + 2
    end_y = pos[1] + text_size[0][1] - 2
    cv2.rectangle(img, pos, (end_x, end_y), bg_color, cv2.FILLED)
    cv2.putText(img, text, pos, cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,0), 1, cv2.LINE_AA)

In [ ]:
cv2.getTextSize('text', cv2.FONT_HERSHEY_COMPLEX, 1, cv2.FILLED)

In [ ]:
cap = cv2.VideoCapture(0)

In [ ]:
while True:
    ret, frame = cap.read()

    img = cv2.resize(frame, (224, 224))
    y_pred = detect_face_mask(img)
    
    if y_pred == 0:
        draw_label(frame, "Mask Detect", (30, 30), (0, 255, 0))
    else:
        draw_label(frame, "No Mask", (30, 30), (0,0, 255))
    cv2.imshow("window", frame)
    if cv2.waitKey(1) and  0xFF == ord('x'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
while True:
    ret, frame = cap.read()
    cood = detect_face(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
    
    for x, y, w, h in cood:
        cv2.rectangle(frame, (x+y), (x+w, y+h), (255, 0, 0),3)
    cv2.imshow("window", frame)
    if cv2.waitKey(1) and  0xFF == ord('x'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
haar = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
def detect_face(img):
    coor = haar.detectMultiScale(img)
    return coor